Lecture 13: HMMs (part 2), decoding (brute force, greedy)
===============

10/30/2023, CS 4/6120 Natural Language Processing, Muzny


Task 1: brute force for "Janet will back the bill"
-----------------

The tables and examples here correspond to section 8.4.6 in the textbook and tables 8.12 and 8.13.

In [1]:
from collections import Counter

In [3]:
# First, set up our model's paramters
keys = "NNP MD VB JJ NN RB DT".split()  # these are our states
values = "0.2767 0.0006 0.0031 0.0453 0.0449 0.0510 0.2026".split()  # this is pi
pi = {keys[i]: float(values[i]) for i in range(len(keys))}
Q = set(keys)
print(pi)
print(Q)


{'NNP': 0.2767, 'MD': 0.0006, 'VB': 0.0031, 'JJ': 0.0453, 'NN': 0.0449, 'RB': 0.051, 'DT': 0.2026}
{'RB', 'NN', 'JJ', 'NNP', 'VB', 'MD', 'DT'}


In [5]:
A_table = ["0.3777 0.0110 0.0009 0.0084 0.0584 0.0090 0.0025".split(),\
           "0.0008 0.0002 0.7968 0.0005 0.0008 0.1698 0.0041".split(),\
          "0.0322 0.0005 0.0050 0.0837 0.0615 0.0514 0.2231".split(),\
          "0.0366 0.0004 0.0001 0.0733 0.4509 0.0036 0.0036".split(),\
          "0.0096 0.0176 0.0014 0.0086 0.1216 0.0177 0.0068".split(),\
          "0.0068 0.0102 0.1011 0.1012 0.0120 0.0728 0.0479".split(),\
          "0.1147 0.0021 0.0002 0.2157 0.4744 0.0102 0.0017".split()]

# p(t_i| t_{i - 1})

# to access a given probability, we'll enter
# "state" and "previous state" in "reverse order"
# consistent with rows being the previous/given state and
# columns being the current one
# A[t_i - 1][t_i]
A = {keys[i]:Counter({keys[j]:float(A_table[i][j]) for j in range(len(keys))}) for i in range(len(keys))}
print(A)

{'NNP': Counter({'NNP': 0.3777, 'NN': 0.0584, 'MD': 0.011, 'RB': 0.009, 'JJ': 0.0084, 'DT': 0.0025, 'VB': 0.0009}), 'MD': Counter({'VB': 0.7968, 'RB': 0.1698, 'DT': 0.0041, 'NNP': 0.0008, 'NN': 0.0008, 'JJ': 0.0005, 'MD': 0.0002}), 'VB': Counter({'DT': 0.2231, 'JJ': 0.0837, 'NN': 0.0615, 'RB': 0.0514, 'NNP': 0.0322, 'VB': 0.005, 'MD': 0.0005}), 'JJ': Counter({'NN': 0.4509, 'JJ': 0.0733, 'NNP': 0.0366, 'RB': 0.0036, 'DT': 0.0036, 'MD': 0.0004, 'VB': 0.0001}), 'NN': Counter({'NN': 0.1216, 'RB': 0.0177, 'MD': 0.0176, 'NNP': 0.0096, 'JJ': 0.0086, 'DT': 0.0068, 'VB': 0.0014}), 'RB': Counter({'JJ': 0.1012, 'VB': 0.1011, 'RB': 0.0728, 'DT': 0.0479, 'NN': 0.012, 'MD': 0.0102, 'NNP': 0.0068}), 'DT': Counter({'NN': 0.4744, 'JJ': 0.2157, 'NNP': 0.1147, 'RB': 0.0102, 'MD': 0.0021, 'DT': 0.0017, 'VB': 0.0002})}


In [7]:
print("p(VB|MD):", A["MD"]["VB"])

# Any experiments that you want here
print("p(NN|JJ)", A["NN"]["JJ"])

p(VB|MD): 0.7968
p(NN|JJ) 0.0086


1. What is the probability of the next word being a noun (`NN`) if we just saw an adjective (`JJ`)? 0.0086

In [8]:
# p(w_i| t_i)

# to access a given probability, we'll enter
# "state" and "word" in "reverse order"
# consistent with rows being the given state and
# columns being the current word
# B[t_i][w_i]
B = {"NNP": Counter({"Janet":0.000032, "the": 0.000048}),\
     "MD": Counter({"will":0.308431}),\
    "VB": Counter({"will":0.000028, "back": 0.000672, "bill":0.000028}),\
    "JJ": Counter({"back":0.00034}),\
    "NN": Counter({"will":0.0002, "back": 0.000223, "bill": 0.002337}),\
    "RB": Counter({"back":0.010446}),\
    "DT": Counter({"the":0.506099})}


In [10]:
print("p(will|MD):", B["MD"]["will"])

# Any experiments that you want here
print("p(back|VB):", B["VB"]["back"])

p(will|MD): 0.308431
p(back|VB): 0.000672


2. What is the probability of the word being "back" if the tag is verb (`VB`)? 0.000672

In [15]:
# so that we can enumerate all the sequences
import itertools


def brute_force(o: list, pi: dict, A: dict, B: dict, Q: set) -> (tuple, float):
    """
    Enumerate all possible tag sequences and choose the best one according to
    the passed in transition and emission tables.
    
    params:
    o - a list of tokens (observations)
    pi - a dictionary of the initial probabilities for all states (p(state | <s>))
    A - a dictionary of transition probabilities where previous state maps to current state (p(t_i| t_i - 1))
    B - a dictionary of emission probabilities where state maps to words (p(w_i | t_i))
    Q - a set of states (strings)
    
    return:
    max_seq - a tuple of the best sequence of tags for the observation
    max_prob - a float of the final calculated probability for that sequence of tags
    """
    # generate the sequences
    # this is the cartesian product
    sequences = list(itertools.product(Q, repeat=len(o)))
    print("Number of states:", len(Q))
    print("Number of words:", len(o))
    print("Number of sequences:", len(Q) ** len(o))
    print("I have generated:", len(sequences))
    
    probabilities = {}
    for seq in sequences:
        
        # TODO: evaluate the probability of this sequence

        # you'll probably want to loop over the sequence
        prob = 1
        for t in range(len(seq)):
            state = seq[t]
            word = o[t]
            emission = B[state][word]
            if t == 0:
                # initial probability
                # you'll need something different for the else statement
                # you will be using A instead of pi in the else statement
                initial_prob = pi[state]
                # print("p(will|MD):", B["MD"]["will"])
             
                prob *= initial_prob * emission
            else:
                # you need to implement the else-case
                
            
                # you'll need the numbers:
                # emission: P(w_i | t_i)
                # pi (if t is 0)
                # transition: p(t_i | t_i - 1) ( if t is > 0)
                transition = A[seq[t-1]][state]
                prob = transition * emission

                # then you'll need to accumulate your product
            
        # finally, link this probability to this sequence
        probabilities[seq] = prob
        
    # find the max probability
    max_prob = max(probabilities.values())
    # find the maximum sequence (the argmax)
    max_seq = max(probabilities, key=probabilities.get, default=())
    return (max_seq, max_prob)

In [17]:
import time


# do a few experiments to get to know the time 
# and how many sequences are being considered here
example = "back the bill".split() 
start = time.time()
print(brute_force(example, pi, A, B, Q))
end = time.time()
print("That took: ", end - start)

Number of states: 7
Number of words: 3
Number of sequences: 343
I have generated: 343
(('RB', 'DT', 'NN'), 0.0011086728)
That took:  0.0005528926849365234


3. How long does it take to perform __brute force__ decoding with a set of 7 states and 3 words (e.g. "back the bill") on your machine? __YOUR ANSWER HERE__

3. How long does it take to perform __brute force__ decoding with a set of 7 states and 5 words on your machine? __YOUR ANSWER HERE__

Task 2: greedy approach for "Janet will back the bill"
-----------------

In [22]:
def greedy(o: list, pi: dict, A: dict, B: dict, Q: set) -> (tuple, float):
    """
    Greedily decode the sequence of tags to go along with a sequence of words
    
    params:
    o - a list of tokens (observations)
    pi - a dictionary of the initial probabilities for all states (p(state | <s>))
    A - a dictionary of transition probabilities where previous state maps to current state (p(t_i| t_i - 1))
    B - a dictionary of emission probabilities where state maps to words (p(w_i | t_i))
    Q - a set of states (strings)
    
    return:
    max_seq - a tuple of the best sequence of tags for the observation
    max_prob - a float of the final calculated probability for that sequence of tags
    """

    
    greedy_seq = []
    for t in range(len(o)):
        # current observation/word
        o_t = o[t]
        word = o[t]
        
        probabilities = {}
        # consider every state
        for state in Q:
            
            # P(w_i | t_i)
            # TODO: get the emission probability for the word given the state
            emission =  B[state][word]
        
            # see if we're considering the first word
            if t == 0:
                prob = pi[state] * emission
                probabilities[state] = prob
            else:
                # consider ONLY THE BEST previous state
                # as the state that we came from 
                # (this is what makes this strategy different than 
                # the Viterbi algorithm, a dynamic programming approach)
                # TODO: get the appropriate transition probability for this state
                transition = A[greedy_seq[-1]][state]
                    
                # calculate the new probability
                # TODO FILL THIS IN
                prob = emission * transition
                probabilities[state] = prob
                
        # choose the best state for the prev state
        max_prob = max(probabilities.values())
        # get the argmax
        max_state = max(probabilities, key=probabilities.get, default=())
        # build up our sequence
        greedy_seq.append(max_state)
        
    return (tuple(greedy_seq), max_prob)

In [23]:
# do a few experiments to get to know the time 
# and how many sequences are being considered here
example = "Janet will back the bill".split() 
start = time.time()
print(greedy(example, pi, A, B, Q))
end = time.time()
print("That took: ", end - start)

(('NNP', 'MD', 'RB', 'DT', 'NN'), 0.0011086728)
That took:  0.00012803077697753906


5. How does the time of your `greedy` approach compare to the `brute_force` approach? __YOUR ANSWER HERE__

6. How about the calculated probability of the final sequence of tags chosen? __YOUR ANSWER HERE__

Task 3: Using `nltk` to tag
-----------

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

In [ ]:
nltk.pos_tag(nltk.word_tokenize("Janet will back the bill"))

7. Does `nltk` agree with your brute force or greedy strategies? __YOUR ANSWER HERE__